In [1]:
import torch
from transformers import MT5ForConditionalGeneration, MT5Config, MT5EncoderModel, MT5Tokenizer, Trainer, TrainingArguments
from progeny_tokenizer import TAPETokenizer
import numpy as np
import math
import random
import scipy
import time
import pandas as pd
from torch.utils.data import DataLoader, RandomSampler, Dataset, BatchSampler
import typing
from pathlib import Path
import argparse
from collections import OrderedDict
import pickle
import pathlib

import matplotlib.pyplot as plt

In [2]:
noperturb_results_tsv = 'generated_seqs/congen_SST5/SST5_leave4out3keep200_t5base_clspool_waeDeterencStart4kstep512dim_cyccon1Start4kstep_lre-04_25ep_noperturb_trainlabel2geninput/16xgen25kunique_SST5_leave4out3keep200_t5base_clspool_waeDeterencStart4kstep512dim_cyccon1Start4kstep_lre-04_25ep_noperturb-congen_seqs25000.tsv'

In [3]:
perturb_results_tsvs = [
    'generated_seqs/congen_SST5/SST5_leave4out3keep200_t5base_clspool_waeDeterencStart4kstep512dim_cyccon1Start4kstep_lre-04_25ep_perturb0029_trainlabel2geninput/16xgen25kunique_SST5_leave4out3keep200_t5base_clspool_waeDeterencStart4kstep512dim_cyccon1Start4kstep_lre-04_25ep_perturb0029-congen_seqs25000.tsv',
    'generated_seqs/congen_SST5/SST5_leave4out3keep200_t5base_clspool_waeDeterencStart4kstep512dim_cyccon1Start4kstep_lre-04_25ep_perturb0059_trainlabel2geninput/16xgen25kunique_SST5_leave4out3keep200_t5base_clspool_waeDeterencStart4kstep512dim_cyccon1Start4kstep_lre-04_25ep_perturb0059-congen_seqs25000.tsv',
    'generated_seqs/congen_SST5/SST5_leave4out3keep200_t5base_clspool_waeDeterencStart4kstep512dim_cyccon1Start4kstep_lre-04_25ep_perturb015_trainlabel2geninput/16xgen25kunique_SST5_leave4out3keep200_t5base_clspool_waeDeterencStart4kstep512dim_cyccon1Start4kstep_lre-04_25ep_perturb015-congen_seqs25000.tsv',
    'generated_seqs/congen_SST5/SST5_leave4out3keep200_t5base_clspool_waeDeterencStart4kstep512dim_cyccon1Start4kstep_lre-04_25ep_perturb029_trainlabel2geninput/16xgen25kunique_SST5_leave4out3keep200_t5base_clspool_waeDeterencStart4kstep512dim_cyccon1Start4kstep_lre-04_25ep_perturb029-congen_seqs25000.tsv',
    'generated_seqs/congen_SST5/SST5_leave4out3keep200_t5base_clspool_waeDeterencStart4kstep512dim_cyccon1Start4kstep_lre-04_25ep_perturb074_trainlabel2geninput/16xgen25kunique_SST5_leave4out3keep200_t5base_clspool_waeDeterencStart4kstep512dim_cyccon1Start4kstep_lre-04_25ep_perturb074-congen_seqs25000.tsv',
    'generated_seqs/congen_SST5/SST5_leave4out3keep200_t5base_clspool_waeDeterencStart4kstep512dim_cyccon1Start4kstep_lre-04_25ep_perturb147_trainlabel2geninput/16xgen25kunique_SST5_leave4out3keep200_t5base_clspool_waeDeterencStart4kstep512dim_cyccon1Start4kstep_lre-04_25ep_perturb147-congen_seqs25000.tsv'

]

In [4]:
noperturb_df = pd.read_table(noperturb_results_tsv)

In [5]:
noperturb_df = noperturb_df.sort_values(by='latent_head_pred', ascending=False)
noperturb_df

,disc_pred,latent_head_pred,gen_input_seq_class,gt_class_pred,gt_highest_prob,gt_2class_pred,gt_neg_prob,gt_pos_prob,generated_seq_ppl,input_seq_ppl,generated_seq,gen_input_seq,repeated_gen,in_train_data_gen
0,5.109805,6.511142,2,3,0.999928,1,0.000026,0.999948,146.135635,231.790436,An often enjoyable romp through the great times.,An often unfunny romp .,False,False
1,5.353800,6.393686,2,3,0.999948,1,0.000024,0.999957,76.394547,231.790436,"An enjoyable, often unfunny romp; it also brin...",An often unfunny romp .,False,False
2,6.194846,6.349895,2,4,0.999931,1,0.000034,0.999938,43.070000,33.838017,"It's a dazzling, nostalgic flick that helps yo...",It 's mindless junk like this that makes you a...,False,False
3,5.524262,6.346447,2,4,0.999921,1,0.000029,0.999931,124.248932,60.907505,We could see the dramatic power of the Winds O...,I wish Windtalkers had had more faith in the d...,False,False
4,5.014483,6.275188,2,4,0.999639,1,0.000050,0.999933,130.631607,99.493553,Consider it a powerful affirmation.,Consider this review life-affirming .,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25029,3.556965,-4.759436,2,0,0.999122,0,0.999946,0.000041,187.167450,70.866104,"Yep, it's a Reno-Yawn. '","Yep , it 's '' Waking up in Reno . ''",False,False
25030,1.112146,-4.782455,2,1,0.893337,0,0.893398,0.000105,133.689987,158.049561,Scratch one as crafted as Scratch : A rip-off ...,A case in point : Doug Pray 's Scratch .,False,False
25031,1.777820,-5.052963,2,1,0.999859,0,0.999984,0.000013,77.432808,45.134209,"It looks like an easy-to-hold, sordid video th...",The thing looks like a made-for-home-video qui...,False,False
25032,2.842080,-5.070903,2,1,0.997833,0,0.999964,0.000026,18.683876,33.218781,A by-the-numbers rip-off of the 20th Century.,An unintentional parody of every teen movie ma...,False,False


In [6]:
noperturb_df['sent_delta'] = noperturb_df['gt_class_pred'] - noperturb_df['gen_input_seq_class']
noperturb_df

,disc_pred,latent_head_pred,gen_input_seq_class,gt_class_pred,gt_highest_prob,gt_2class_pred,gt_neg_prob,gt_pos_prob,generated_seq_ppl,input_seq_ppl,generated_seq,gen_input_seq,repeated_gen,in_train_data_gen,sent_delta
0,5.109805,6.511142,2,3,0.999928,1,0.000026,0.999948,146.135635,231.790436,An often enjoyable romp through the great times.,An often unfunny romp .,False,False,1
1,5.353800,6.393686,2,3,0.999948,1,0.000024,0.999957,76.394547,231.790436,"An enjoyable, often unfunny romp; it also brin...",An often unfunny romp .,False,False,1
2,6.194846,6.349895,2,4,0.999931,1,0.000034,0.999938,43.070000,33.838017,"It's a dazzling, nostalgic flick that helps yo...",It 's mindless junk like this that makes you a...,False,False,2
3,5.524262,6.346447,2,4,0.999921,1,0.000029,0.999931,124.248932,60.907505,We could see the dramatic power of the Winds O...,I wish Windtalkers had had more faith in the d...,False,False,2
4,5.014483,6.275188,2,4,0.999639,1,0.000050,0.999933,130.631607,99.493553,Consider it a powerful affirmation.,Consider this review life-affirming .,False,False,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25029,3.556965,-4.759436,2,0,0.999122,0,0.999946,0.000041,187.167450,70.866104,"Yep, it's a Reno-Yawn. '","Yep , it 's '' Waking up in Reno . ''",False,False,-2
25030,1.112146,-4.782455,2,1,0.893337,0,0.893398,0.000105,133.689987,158.049561,Scratch one as crafted as Scratch : A rip-off ...,A case in point : Doug Pray 's Scratch .,False,False,-1
25031,1.777820,-5.052963,2,1,0.999859,0,0.999984,0.000013,77.432808,45.134209,"It looks like an easy-to-hold, sordid video th...",The thing looks like a made-for-home-video qui...,False,False,-1
25032,2.842080,-5.070903,2,1,0.997833,0,0.999964,0.000026,18.683876,33.218781,A by-the-numbers rip-off of the 20th Century.,An unintentional parody of every teen movie ma...,False,False,-1


In [7]:
noperturb_df['ppl_delta'] = noperturb_df['generated_seq_ppl'] - noperturb_df['input_seq_ppl']
noperturb_df

,disc_pred,latent_head_pred,gen_input_seq_class,gt_class_pred,gt_highest_prob,gt_2class_pred,gt_neg_prob,gt_pos_prob,generated_seq_ppl,input_seq_ppl,generated_seq,gen_input_seq,repeated_gen,in_train_data_gen,sent_delta,ppl_delta
0,5.109805,6.511142,2,3,0.999928,1,0.000026,0.999948,146.135635,231.790436,An often enjoyable romp through the great times.,An often unfunny romp .,False,False,1,-85.654800
1,5.353800,6.393686,2,3,0.999948,1,0.000024,0.999957,76.394547,231.790436,"An enjoyable, often unfunny romp; it also brin...",An often unfunny romp .,False,False,1,-155.395889
2,6.194846,6.349895,2,4,0.999931,1,0.000034,0.999938,43.070000,33.838017,"It's a dazzling, nostalgic flick that helps yo...",It 's mindless junk like this that makes you a...,False,False,2,9.231983
3,5.524262,6.346447,2,4,0.999921,1,0.000029,0.999931,124.248932,60.907505,We could see the dramatic power of the Winds O...,I wish Windtalkers had had more faith in the d...,False,False,2,63.341427
4,5.014483,6.275188,2,4,0.999639,1,0.000050,0.999933,130.631607,99.493553,Consider it a powerful affirmation.,Consider this review life-affirming .,False,False,2,31.138054
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25029,3.556965,-4.759436,2,0,0.999122,0,0.999946,0.000041,187.167450,70.866104,"Yep, it's a Reno-Yawn. '","Yep , it 's '' Waking up in Reno . ''",False,False,-2,116.301346
25030,1.112146,-4.782455,2,1,0.893337,0,0.893398,0.000105,133.689987,158.049561,Scratch one as crafted as Scratch : A rip-off ...,A case in point : Doug Pray 's Scratch .,False,False,-1,-24.359573
25031,1.777820,-5.052963,2,1,0.999859,0,0.999984,0.000013,77.432808,45.134209,"It looks like an easy-to-hold, sordid video th...",The thing looks like a made-for-home-video qui...,False,False,-1,32.298599
25032,2.842080,-5.070903,2,1,0.997833,0,0.999964,0.000026,18.683876,33.218781,A by-the-numbers rip-off of the 20th Century.,An unintentional parody of every teen movie ma...,False,False,-1,-14.534904


# Compute % target class for noperturb

In [8]:
target_classes = [3,4]
topk_list = [25000, 10000, 1000, 100, 10]

percent_target_class = []

In [9]:
gt_class_preds = noperturb_df['gt_class_pred']
gen_input_seq_classes = noperturb_df['gen_input_seq_class']
sent_deltas = noperturb_df['sent_delta']

generated_seq_ppls = noperturb_df['generated_seq_ppl']
input_seq_ppls = noperturb_df['input_seq_ppl']
ppl_deltas = noperturb_df['ppl_delta']

In [10]:
df = noperturb_df

In [11]:
for target_class in target_classes:
    total_num  = len(df['gt_class_pred'])
    print("target_class: ", target_class)
    num_target_class = np.sum(df['gt_class_pred'] == target_class)
    
    percent_target_class = num_target_class / total_num *100
    print("percent_target_class: ", percent_target_class)
    
    for topk in topk_list:
        topk_gt_class_preds = gt_class_preds[:topk]
        topk_sent_deltas = sent_deltas[:topk]
        
        
        topk_num  = len(topk_gt_class_preds)
        print("topk: ", topk)
        topk_num_target_class = np.sum(topk_gt_class_preds == target_class)
        topk_percent_target_class = topk_num_target_class / topk_num *100
        print("topk_percent_target_class: ", topk_percent_target_class)
    
        topk_sent_delta_mean = np.mean(topk_sent_deltas)
        print("topk_sent_delta_mean: ", topk_sent_delta_mean)
        
        print("*")
    print("--------------")
    
    
print("-------For all target classes-------")
print("target_classes: ", target_classes)
total_num  = len(df['gt_class_pred'])
num_target_class = np.sum(df['gt_class_pred'].isin(target_classes))

percent_target_class = num_target_class / total_num *100
print("percent_target_class: ", percent_target_class)

for topk in topk_list:
    topk_gt_class_preds = gt_class_preds[:topk]
    topk_sent_deltas = sent_deltas[:topk]
    
    topk_generated_seq_ppls = generated_seq_ppls[:topk]
    topk_ppl_deltas = ppl_deltas[:topk]

    topk_num  = len(topk_gt_class_preds)
    print("topk: ", topk)
    topk_num_target_class = np.sum(topk_gt_class_preds.isin(target_classes))
    topk_percent_target_class = topk_num_target_class / topk_num *100
    print("topk_percent_target_class: ", topk_percent_target_class)

    topk_sent_delta_mean = np.mean(topk_sent_deltas)
    print("topk_sent_delta_mean: ", topk_sent_delta_mean)
    
    
    topk_generated_seq_ppl_mean = np.mean(topk_generated_seq_ppls)
    topk_generated_seq_ppl_std = np.std(topk_generated_seq_ppls)
    print("topk_generated_seq_ppl_mean: ", topk_generated_seq_ppl_mean)
    print("topk_generated_seq_ppl_std: ", topk_generated_seq_ppl_std)
    
    topk_ppl_delta_mean = np.mean(topk_ppl_deltas)
    topk_ppl_delta_std = np.std(topk_ppl_deltas)
    print("topk_ppl_delta_mean: ", topk_ppl_delta_mean)
    print("topk_ppl_delta_std: ", topk_ppl_delta_std)

    print("*")

target_class:  3
percent_target_class:  43.74850203722937
topk:  25000
topk_percent_target_class:  43.808
topk_sent_delta_mean:  0.58348
*
topk:  10000
topk_percent_target_class:  61.44
topk_sent_delta_mean:  1.0968
*
topk:  1000
topk_percent_target_class:  54.2
topk_sent_delta_mean:  1.394
*
topk:  100
topk_percent_target_class:  51.0
topk_sent_delta_mean:  1.47
*
topk:  10
topk_percent_target_class:  50.0
topk_sent_delta_mean:  1.5
*
--------------
target_class:  4
percent_target_class:  11.700087880482544
topk:  25000
topk_percent_target_class:  11.716
topk_sent_delta_mean:  0.58348
*
topk:  10000
topk_percent_target_class:  24.23
topk_sent_delta_mean:  1.0968
*
topk:  1000
topk_percent_target_class:  42.6
topk_sent_delta_mean:  1.394
*
topk:  100
topk_percent_target_class:  48.0
topk_sent_delta_mean:  1.47
*
topk:  10
topk_percent_target_class:  50.0
topk_sent_delta_mean:  1.5
*
--------------
-------For all target classes-------
target_classes:  [3, 4]
percent_target_class:  55.44

# iterate through all perturbed result tsv files

In [12]:
for perturb_results_tsv in perturb_results_tsvs:
    print("*-"*30)
    print("perturb_results_tsv: ", perturb_results_tsv)
    
    perturb_df = pd.read_table(perturb_results_tsv)
    perturb_df = perturb_df.sort_values(by='latent_head_pred', ascending=False)
    perturb_df['sent_delta'] = perturb_df['gt_class_pred'] - perturb_df['gen_input_seq_class']
    perturb_df['ppl_delta'] = perturb_df['generated_seq_ppl'] - perturb_df['input_seq_ppl']

    gt_class_preds = perturb_df['gt_class_pred']
    gen_input_seq_classes = perturb_df['gen_input_seq_class']
    sent_deltas = perturb_df['sent_delta']
    
    generated_seq_ppls = perturb_df['generated_seq_ppl']
    input_seq_ppls = perturb_df['input_seq_ppl']
    ppl_deltas = perturb_df['ppl_delta']

    for target_class in target_classes:
        total_num  = len(perturb_df['gt_class_pred'])
        print("target_class: ", target_class)
        num_target_class = np.sum(perturb_df['gt_class_pred'] == target_class)

        percent_target_class = num_target_class / total_num *100
        print("percent_target_class: ", percent_target_class)

        for topk in topk_list:
            topk_gt_class_preds = gt_class_preds[:topk]
            topk_sent_deltas = sent_deltas[:topk]


            topk_num  = len(topk_gt_class_preds)
            print("topk: ", topk)
    #         print("topk_gt_class_preds: ", topk_gt_class_preds)
            topk_num_target_class = np.sum(topk_gt_class_preds == target_class)
            topk_percent_target_class = topk_num_target_class / topk_num *100
#             print("topk_num_target_class: ", topk_num_target_class)
#             print("topk_num: ", topk_num)
            print("topk_percent_target_class: ", topk_percent_target_class)

            topk_sent_delta_mean = np.mean(topk_sent_deltas)
    #         print("topk_sent_deltas: ", topk_sent_deltas)
            print("topk_sent_delta_mean: ", topk_sent_delta_mean)

            print("*")
        print("--------------")


    print("-------For all target classes-------")
    print("target_classes: ", target_classes)
    total_num  = len(perturb_df['gt_class_pred'])
    num_target_class = np.sum(perturb_df['gt_class_pred'].isin(target_classes))

    percent_target_class = num_target_class / total_num *100
    print("percent_target_class: ", percent_target_class)

    for topk in topk_list:
        topk_gt_class_preds = gt_class_preds[:topk]
        topk_sent_deltas = sent_deltas[:topk]
    
        topk_generated_seq_ppls = generated_seq_ppls[:topk]
        topk_ppl_deltas = ppl_deltas[:topk]

        topk_num  = len(topk_gt_class_preds)
        print("topk: ", topk)
    #         print("topk_gt_class_preds: ", topk_gt_class_preds)
        topk_num_target_class = np.sum(topk_gt_class_preds.isin(target_classes))
        topk_percent_target_class = topk_num_target_class / topk_num *100
#         print("topk_num_target_class: ", topk_num_target_class)
#         print("topk_num: ", topk_num)
        print("topk_percent_target_class: ", topk_percent_target_class)

        topk_sent_delta_mean = np.mean(topk_sent_deltas)
    #         print("topk_sent_deltas: ", topk_sent_deltas)
        print("topk_sent_delta_mean: ", topk_sent_delta_mean)

    
        topk_generated_seq_ppl_mean = np.mean(topk_generated_seq_ppls)
        topk_generated_seq_ppl_std = np.std(topk_generated_seq_ppls)
        print("topk_generated_seq_ppl_mean: ", topk_generated_seq_ppl_mean)
        print("topk_generated_seq_ppl_std: ", topk_generated_seq_ppl_std)

        topk_ppl_delta_mean = np.mean(topk_ppl_deltas)
        topk_ppl_delta_std = np.std(topk_ppl_deltas)
        print("topk_ppl_delta_mean: ", topk_ppl_delta_mean)
        print("topk_ppl_delta_std: ", topk_ppl_delta_std)

        print("*")

*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
perturb_results_tsv:  generated_seqs/congen_SST5/SST5_leave4out3keep200_t5base_clspool_waeDeterencStart4kstep512dim_cyccon1Start4kstep_lre-04_25ep_perturb0029_trainlabel2geninput/16xgen25kunique_SST5_leave4out3keep200_t5base_clspool_waeDeterencStart4kstep512dim_cyccon1Start4kstep_lre-04_25ep_perturb0029-congen_seqs25000.tsv
target_class:  3
percent_target_class:  44.92602958816473
topk:  25000
topk_percent_target_class:  44.944
topk_sent_delta_mean:  0.62608
*
topk:  10000
topk_percent_target_class:  61.519999999999996
topk_sent_delta_mean:  1.1291
*
topk:  1000
topk_percent_target_class:  51.6
topk_sent_delta_mean:  1.446
*
topk:  100
topk_percent_target_class:  46.0
topk_sent_delta_mean:  1.54
*
topk:  10
topk_percent_target_class:  80.0
topk_sent_delta_mean:  1.2
*
--------------
target_class:  4
percent_target_class:  12.906837265093962
topk:  25000
topk_percent_target_class:  12.912
topk_sent_delta_mean:  0.62608
*
topk:

target_class:  3
percent_target_class:  51.11448430135016
topk:  25000
topk_percent_target_class:  51.184
topk_sent_delta_mean:  0.71152
*
topk:  10000
topk_percent_target_class:  61.21
topk_sent_delta_mean:  1.0536
*
topk:  1000
topk_percent_target_class:  53.300000000000004
topk_sent_delta_mean:  1.415
*
topk:  100
topk_percent_target_class:  44.0
topk_sent_delta_mean:  1.52
*
topk:  10
topk_percent_target_class:  50.0
topk_sent_delta_mean:  1.5
*
--------------
target_class:  4
percent_target_class:  12.151473995366302
topk:  25000
topk_percent_target_class:  12.168
topk_sent_delta_mean:  0.71152
*
topk:  10000
topk_percent_target_class:  22.27
topk_sent_delta_mean:  1.0536
*
topk:  1000
topk_percent_target_class:  44.1
topk_sent_delta_mean:  1.415
*
topk:  100
topk_percent_target_class:  54.0
topk_sent_delta_mean:  1.52
*
topk:  10
topk_percent_target_class:  50.0
topk_sent_delta_mean:  1.5
*
--------------
-------For all target classes-------
target_classes:  [3, 4]
percent_target